In [6]:
import urllib.request as req
import zipfile
import os.path
import MeCab
from gensim import models

#MeCabの初期化
mecab = MeCab.Tagger()
mecab.parse("")

#保存したDoc2Vec学習モデルを読み込み
model = models.Doc2Vec.load('aozora.model')

#分類用のZIPファイルを開き、中の文書を取得する
def read_book(url, zipname):
    if not os.path.exists(zipname):
        req.urlretrieve(url, zipname)
        
    with zipfile.ZipFile(zipname, "r") as zf:
        for filename in zf.namelist():
            with zf.open(filename, "r") as f:
                return f.read().decode("shift-jis")
            
#引数のテキストを分かち書きにして配列にする
def split_words(text):
    node = mecab.parseToNode(text)
    wakati_words = []
    while node is not None:
        hinshi = node.feature.split(",")[0]
        if hinshi in ["名詞"]:
            wakati_words.append(node.surface)
        elif hinshi in ["動詞", "形容詞"]:
            wakati_words.append(node.feature.split(",")[6])
        node = node.next
    return wakati_words

#引数のタイトル、URLの作品を分類する
def similar(title, url):
    zipname = url.split("/")[-1]
    
    words = read_book(url, zipname)
    wakati_words = split_words(words)
    vector = model.infer_vector(wakati_words)
    print("---「" + title +'」と似た作品は？---')
    print(model.dv.most_similar([vector], topn=3))
    print("")
    
#各作家の作品を１つずつ分類
similar("宮沢 賢治 : よだかの星",
       "https://www.aozora.gr.jp/cards/000081/files/473_ruby_467.zip")

similar("芥川 龍之介 : 犬と猫",
        "https://www.aozora.gr.jp/cards/000879/files/56_ruby_845.zip")

similar("ボー エドガー・アラン : マリー・ロジェエの怪事件",
        "https://www.aozora.gr.jp/cards/000094/files/4261_ruby_54182.zip")

similar("紫式部 : 源氏物語 06 末摘花",
        "https://www.aozora.gr.jp/cards/000052/files/5021_ruby_11106.zip")

---「宮沢 賢治 : よだかの星」と似た作品は？---
[('宮沢\u3000賢治:やまなし', 0.9115085601806641), ('宮沢\u3000賢治:セロ弾きのゴーシュ', 0.8437379598617554), ('宮沢\u3000賢治:どんぐりと山猫', 0.8322207927703857)]

---「芥川 龍之介 : 犬と猫」と似た作品は？---
[('芥川\u3000龍之介:老年', 0.8338056206703186), ('宮沢\u3000賢治:注文の多い料理店', 0.7239671349525452), ('紫式部:源氏物語 05 若紫', 0.709602415561676)]

---「ボー エドガー・アラン : マリー・ロジェエの怪事件」と似た作品は？---
[('ポー\u3000エドガー・アラン:ウィリアム・ウィルスン', 0.8877441883087158), ('ポー\u3000エドガー・アラン:黒猫', 0.816244900226593), ('ポー\u3000エドガー・アラン:落穴と振子', 0.7044992446899414)]

---「紫式部 : 源氏物語 06 末摘花」と似た作品は？---
[('紫式部:源氏物語 05 若紫', 0.9214043617248535), ('紫式部:源氏物語 03 空蝉', 0.8729607462882996), ('紫式部:源氏物語 02 帚木', 0.8413125872612)]

